# Expand the Heat Wave and Cold Snap Libraries


The heat wave and cold snap libraries are based on data provided by Alfred Wan on 10-Nov 2025. He used a definition that aligns with the draft NERC TPL-008-1 standard. Heat waves are events in which the 3-day rolling average of daily maximum temperature exceeds the 97.5 percentile. Cold snaps which are based on 3-day rolling average of daily minimum temperatures below the 2.5 percentile. County-level temperatures were population-weighted in each TPL-08 region to create hourly time series by region.

This code formats the library into a clean .csv file and adds in regional metadata. It also calculates and merges in information about the peak load (net and total) during each event.

In [1]:
# Start by importing the packages we need:
import os
import warnings

import pandas as pd
import numpy as np

from datetime import timedelta


## Suppress Future Warnings


In [2]:
# Suppress future warnings:
warnings.simplefilter(action='ignore', category=FutureWarning)


## Set the Directory Structure

In [15]:
# Identify the top-level directory and the subdirectory where the data will be stored:
metadata_input_dir =  '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2026_tbd/data/'
events_data_input_dir =  '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2026_tbd/data/thermal_events_data/'
load_data_input_dir =  '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2026_tbd/data/load_data/'
wind_solar_data_input_dir =  '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2026_tbd/data/wind_solar_data/'
data_output_dir = '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2026_tbd/data/thermal_events_data/'


## Write a Function to Merge the Load and Renewables Data Together


In [58]:
def merge_load_wind_solar_data(region: str, load_data_input_dir: str, wind_solar_data_input_dir: str):
    
    # Read in the load data processed by the "aggregate_gridview_load_data.ipynb" notebook:
    load_df = pd.read_csv((load_data_input_dir + 'WECC_Hourly_Loads_1980_to_2025.csv'))

    # Rename the load data for the region of interest:
    load_df.rename(columns={(region + '_Load_MWh'): 'Region_Load_MWh'}, inplace=True) 

    # Only keep the columns we need:
    load_df = load_df[['Time_UTC', 'WECC_Load_MWh', 'Region_Load_MWh']].copy()

    # Convert the time to a datetime variable:
    load_df['Time_UTC'] = pd.to_datetime(load_df['Time_UTC'])
    
    # Read in the wind and solar data processed by the "process_wind_solar_data.ipynb" notebook:
    renew_df = pd.read_csv((wind_solar_data_input_dir + 'WECC_Hourly_Wind_Solar_1980_to_2022.csv'))

    # Rename the wind and solar data for the region of interest:
    renew_df.rename(columns={(region + '_Solar_MWh'): 'Region_Solar_MWh', (region + '_Wind_MWh'): 'Region_Wind_MWh'}, inplace=True) 

    # Only keep the columns we need:
    renew_df = renew_df[['Time_UTC', 'WECC_Solar_MWh', 'WECC_Wind_MWh', 'Region_Solar_MWh', 'Region_Wind_MWh']].copy()

    # Convert the time to a datetime variable:
    renew_df['Time_UTC'] = pd.to_datetime(renew_df['Time_UTC'])

    # Merge the two dataframes together based on common times:
    output_df = load_df.merge(renew_df, on=['Time_UTC'], how='left')

    # Calculate the net load by subtracting available renewable generation from the raw load:
    output_df['WECC_Net_Load_MWh'] = output_df['WECC_Load_MWh'] - output_df['WECC_Solar_MWh'] - output_df['WECC_Wind_MWh']
    output_df['Region_Net_Load_MWh'] = output_df['Region_Load_MWh'] - output_df['Region_Solar_MWh'] - output_df['Region_Wind_MWh']

    # Calculate the total renewables:
    output_df['WECC_Renew_MWh'] = output_df['WECC_Solar_MWh'] + output_df['WECC_Wind_MWh']
    output_df['Region_Renew_MWh'] = output_df['Region_Solar_MWh'] + output_df['Region_Wind_MWh']
    
    # Only keep the columns we need:
    output_df = output_df[['Time_UTC', 'WECC_Load_MWh', 'WECC_Renew_MWh', 'WECC_Net_Load_MWh', 'Region_Load_MWh', 'Region_Renew_MWh', 'Region_Net_Load_MWh']].copy()
    
    return output_df


In [59]:
# Test the function:
output_df = merge_load_wind_solar_data(region = 'CA', 
                                       load_data_input_dir = load_data_input_dir, 
                                       wind_solar_data_input_dir = wind_solar_data_input_dir)

output_df


,Time_UTC,WECC_Load_MWh,WECC_Renew_MWh,WECC_Net_Load_MWh,Region_Load_MWh,Region_Renew_MWh,Region_Net_Load_MWh
0,1980-01-01 00:00:00,95949.05,1249.23,94699.82,34056.59,107.11,33949.48
1,1980-01-01 01:00:00,100408.36,1385.19,99023.17,35669.03,127.62,35541.41
2,1980-01-01 02:00:00,102709.84,1296.80,101413.04,36599.53,94.96,36504.57
3,1980-01-01 03:00:00,104593.05,1270.69,103322.36,37876.25,79.42,37796.83
4,1980-01-01 04:00:00,104588.55,1287.97,103300.58,38507.77,70.93,38436.84
...,...,...,...,...,...,...,...
395275,2024-12-31 19:00:00,97933.90,NaN,NaN,33235.87,NaN,NaN
395276,2024-12-31 20:00:00,97279.37,NaN,NaN,33197.71,NaN,NaN
395277,2024-12-31 21:00:00,97107.29,NaN,NaN,33381.65,NaN,NaN
395278,2024-12-31 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN


## Process the Heat Wave and Cold Snap Library


In [91]:
# Define a function to add new information to the existing HW and CS libraries:
def expand_hw_cs_libraries(hw_cs: str, time_window: int, 
                           metadata_input_dir: str, events_data_input_dir: str, load_data_input_dir: str, wind_solar_data_input_dir: str, data_output_dir: str):

    # Read in NERC region name file and rename some columns:
    nerc = pd.read_csv((metadata_input_dir + 'nerc_tpl08_region_names.csv'))
    nerc.rename(columns={'number': 'NERC', 'short_name': 'Region'}, inplace=True)
        
    if hw_cs == 'HW':
       # Read in the raw data from the heat wave library:
       temp_df = pd.read_csv((events_data_input_dir + 'Tmax_based_heat_wave_library.csv'))
    elif hw_cs == 'CS':
       # Read in the raw data from the cold snap library:
       temp_df = pd.read_csv((events_data_input_dir + 'Tmin_based_cold_snap_library.csv'))
 
    # Rename the TPL-08 regions to match the "short_name" in the nerc_tpl08_region_names.csv file:
    temp_df.loc[(temp_df['NERC_ID'] == 'California'),'Region'] = 'CA'
    temp_df.loc[(temp_df['NERC_ID'] == 'ERCOT'),'Region'] = 'ERCOT'
    temp_df.loc[(temp_df['NERC_ID'] == 'Florida'),'Region'] = 'FL'
    temp_df.loc[(temp_df['NERC_ID'] == 'Great Basin'),'Region'] = 'GB'
    temp_df.loc[(temp_df['NERC_ID'] == 'ISONE'),'Region'] = 'ISONE'
    temp_df.loc[(temp_df['NERC_ID'] == 'Maritimes'),'Region'] = 'MT'
    temp_df.loc[(temp_df['NERC_ID'] == 'MISO-N'),'Region'] = 'MISO-N'
    temp_df.loc[(temp_df['NERC_ID'] == 'MISO-S'),'Region'] = 'MISO-S'
    temp_df.loc[(temp_df['NERC_ID'] == 'NYISO'),'Region'] = 'NYISO'
    temp_df.loc[(temp_df['NERC_ID'] == 'Pacific Northwest'),'Region'] = 'PNW'
    temp_df.loc[(temp_df['NERC_ID'] == 'PJM'),'Region'] = 'PJM'
    temp_df.loc[(temp_df['NERC_ID'] == 'Rocky Mtn'),'Region'] = 'RM'
    temp_df.loc[(temp_df['NERC_ID'] == 'SERC'),'Region'] = 'SERC'
    temp_df.loc[(temp_df['NERC_ID'] == 'Southwest'),'Region'] = 'SW'
    temp_df.loc[(temp_df['NERC_ID'] == 'SPP-N'),'Region'] = 'SPP-N'
    temp_df.loc[(temp_df['NERC_ID'] == 'SPP-S'),'Region'] = 'SPP-S'

    # Merge in the NERC region name:
    temp_df = temp_df.merge(nerc, on='Region', how='left')

    # Rename the temperature variable to something more generic:
    if hw_cs == 'HW':
       temp_df.rename(columns={'highest_temperature': 'T_Max_Min'}, inplace=True)
    elif hw_cs == 'CS':
       temp_df.rename(columns={'lowest_temperature': 'T_Max_Min'}, inplace=True) 

    # Convert the temperatures from Kelvin to Fahrenheit:
    temp_df['T_Max_Min'] = ((1.8 * (temp_df['T_Max_Min'] - 273)) + 32).round(2)
    
    # Only keep the columns we need:
    temp_df = temp_df[['NERC','Region','start_date','end_date','centroid_date','T_Max_Min','duration']].copy()
      
    # Rename the columns for consistency:
    temp_df.rename(columns={'start_date': 'Start', 'end_date': 'End', 'centroid_date': 'Center', 'duration': 'Duration'}, inplace=True)
    
    # Set all dates to datetimes variable and extract the day of year for the centroid date:
    temp_df['Start'] = pd.to_datetime(temp_df['Start'])
    temp_df['End'] = pd.to_datetime(temp_df['End'])
    temp_df['Center'] = pd.to_datetime(temp_df['Center'])
    temp_df['Center_DOY'] = temp_df['Center'].dt.day_of_year

    # Add a new column with a unique row number and an empty column to the dataframe to store the unique ID:
    temp_df['Row_Number'] = np.arange(len(temp_df))
    temp_df['UID_Temp'] = np.nan

    # Make a list of all of the unique NERC regions and definitions in "temp_df":
    nerc_regions = temp_df['NERC'].unique()
    
    # Loop over the unique NERC region and definition combinations:
    for i in range(len(nerc_regions)):
                  
        # Subset to just the data for that unique combination:
        subset_df = temp_df[temp_df['NERC'].isin([nerc_regions[i]])].copy()

        # If the subset isn't empty then assign a unique ID for each event with that combination:
        if subset_df.empty == False:
          
            # Add a new column with a unique row number:
            subset_df['Event_Number'] = np.arange(len(subset_df)) + 1

            # Add a new column with a unique identifier:
            if hw_cs == 'HW': 
               subset_df['UID'] = ('HW_NERC' + subset_df['NERC'].astype(str) + '_Event' + subset_df['Event_Number'].astype(str))
            else:
               subset_df['UID'] = ('CS_NERC' + subset_df['NERC'].astype(str) + '_Event' + subset_df['Event_Number'].astype(str)) 

            # Loop over the subset and assign the UID back into the main dataframe:
            for row in range(len(subset_df)):
                temp_df.loc[temp_df['Row_Number'] == subset_df['Row_Number'].iloc[row], 'UID_Temp'] = subset_df['UID'].iloc[row]

            # Clean up and move to the next combination:
            del subset_df, row

    # Rename the UID variable:
    temp_df.rename(columns={'UID_Temp': 'UID'}, inplace=True)

    # Loop over the rows of the dataframe and find number of corresponding events within X days of the event in a given row:
    for row in range(len(temp_df)):
        # Subset the data to just events in the library whose centroid date is within X days of centroid date for the event in a given row:
        time_subset_df = temp_df[(pd.to_datetime(temp_df['Center']) > (pd.to_datetime(temp_df['Center'].iloc[row]) - pd.Timedelta(time_window, 'd'))) & 
                                 (pd.to_datetime(temp_df['Center']) < (pd.to_datetime(temp_df['Center'].iloc[row]) + pd.Timedelta(time_window, 'd')))].copy()

        # Store the size of that subset:
        if time_subset_df.empty == False:
           temp_df.at[row, 'Regions_Impacted'] = time_subset_df.shape[0]
        else:
           temp_df.at[row, 'Regions_Impacted'] = 1
            
    # Make a copy of the necessary variables to output:
    output_df = temp_df[['UID', 'NERC', 'Region', 'Start', 'End', 'Center', 'Center_DOY', 'T_Max_Min', 'Duration', 'Regions_Impacted']].copy(deep=False)

    # Sort the date by region number and date then reset the index:
    output_df.sort_values(by=['NERC', 'Start'], inplace=True)
    output_df.reset_index(inplace=True, drop=True)

    # Loop over the rows of the dataframe calculate the peak load for the event if the event region is in the WECC:
    for row in range(len(output_df)):
        event_df = output_df.iloc[[row]]
        
        # Extract the event parameters:
        region = event_df['Region'].item()
        start_date = pd.to_datetime(event_df['Start'].item())
        end_date = pd.to_datetime(event_df['End'].item())

        if region == 'CA' or region == 'PNW' or region == 'GB' or region == 'SW' or region == 'RM':
        
            # Process the load data using the function created above:
            load_df = merge_load_wind_solar_data(region = region, 
                                                 load_data_input_dir = load_data_input_dir, 
                                                 wind_solar_data_input_dir = wind_solar_data_input_dir)

            # Subset to just the period of the event being processed:
            load_subset_df = load_df[(load_df['Time_UTC'] >= start_date) & (load_df['Time_UTC'] < (end_date + pd.Timedelta(days=1)))].copy()

            # Calculate the peak load and net load for the WECC and the region:
            wecc_load_peak = load_subset_df['WECC_Load_MWh'].max().round(0)
            wecc_net_load_peak = load_subset_df['WECC_Net_Load_MWh'].max().round(0)
            region_load_peak = load_subset_df['Region_Load_MWh'].max().round(0)
            region_net_load_peak = load_subset_df['Region_Net_Load_MWh'].max().round(0)

            # Add the load results back into the main dataframe:
            output_df.at[row, 'WECC_Load_Peak_MWh'] = wecc_load_peak
            output_df.at[row, 'WECC_Net_Load_Peak_MWh'] = wecc_net_load_peak
            output_df.at[row, 'Region_Load_Peak_MWh'] = region_load_peak
            output_df.at[row, 'Region_Net_Load_Peak_MWh'] = region_net_load_peak

            # Clean up and move to the next event:
            del event_df, region, start_date, end_date, load_df, load_subset_df
        
    # Write out the dataframe to a .csv file:
    if hw_cs == 'HW':
       output_df.to_csv((os.path.join(data_output_dir, 'hw_library_expanded.csv')), sep=',', index=False)
    if hw_cs == 'CS':
       output_df.to_csv((os.path.join(data_output_dir, 'cs_library_expanded.csv')), sep=',', index=False)
        
    return output_df


In [92]:
# Execute the function:
output_df = expand_hw_cs_libraries(hw_cs = 'HW',
                                   time_window = 3, # Time (in +/- days) over which to identify coincident events across NERC regions
                                   metadata_input_dir = metadata_input_dir,
                                   events_data_input_dir = events_data_input_dir, 
                                   load_data_input_dir = load_data_input_dir,
                                   wind_solar_data_input_dir = wind_solar_data_input_dir,
                                   data_output_dir = data_output_dir)

output_df


,UID,NERC,Region,Start,End,Center,Center_DOY,T_Max_Min,Duration,Regions_Impacted,WECC_Load_Peak_MWh,WECC_Net_Load_Peak_MWh,Region_Load_Peak_MWh,Region_Net_Load_Peak_MWh
0,HW_NERC1_Event1,1,CA,1980-07-23,1980-08-02,1980-07-28,210,94.16,11,5.0,162321.0,160115.0,66910.0,65363.0
1,HW_NERC1_Event2,1,CA,1980-08-10,1980-08-11,1980-08-10,223,90.64,2,3.0,155245.0,152573.0,62535.0,60665.0
2,HW_NERC1_Event3,1,CA,1980-09-30,1980-10-03,1980-10-01,275,93.17,4,1.0,137108.0,134952.0,60473.0,59415.0
3,HW_NERC1_Event4,1,CA,1981-08-06,1981-08-09,1981-08-07,219,92.93,4,2.0,154580.0,152824.0,61305.0,60265.0
4,HW_NERC1_Event5,1,CA,1981-08-27,1981-08-28,1981-08-27,239,92.01,2,1.0,152360.0,149052.0,65232.0,63686.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1723,HW_NERC16_Event96,16,SW,2024-06-06,2024-06-07,2024-06-06,158,99.58,2,2.0,139620.0,NaN,30502.0,NaN
1724,HW_NERC16_Event97,16,SW,2024-06-13,2024-06-13,2024-06-13,165,99.01,1,2.0,134146.0,NaN,29646.0,NaN
1725,HW_NERC16_Event98,16,SW,2024-07-03,2024-07-14,2024-07-08,190,101.40,12,8.0,162028.0,NaN,33055.0,NaN
1726,HW_NERC16_Event99,16,SW,2024-08-05,2024-08-05,2024-08-05,218,98.97,1,5.0,155850.0,NaN,31248.0,NaN
